In [ ]:
!pip install tensorflow
!pip install numpy
!pip install mtcnn
!pip install keras_facenet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keras_facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=6103caaaedaa91df25f4074bd965ab442fc9a2a53030d8945f50b7a7be976586
  Stored in directory: /root/.cache/pip/wheels/05/b0/f5/19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras_facenet


In [ ]:
!pip install scikit-learn

In [85]:
import cv2
import numpy as np
import os
from keras_facenet import FaceNet
from sklearn.linear_model import LogisticRegression
from google.colab.patches import cv2_imshow

In [24]:
print("Loading Face Model")
embedder = FaceNet()
print("✅ FaceNet loaded!")

Loading Face Model
✅ FaceNet loaded!


In [25]:
def get_embedding(face):
    """Get 512-dim FaceNet embedding"""
    face = np.expand_dims(face, axis=0)
    embedding = embedder.embeddings(face)
    return embedding[0]

In [71]:
# 2. Initialize OpenCV face detector
def cropFace(img) :
  face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_default.xml')
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  #detect faces
  Faces = face_cascade.detectMultiScale(
      gray,
      minSize=(30,30),
      minNeighbors=5,
      scaleFactor=1.1
  )

  if len(Faces) == 0:
    return None

  # Get the largest face
  Faces = sorted(Faces, key=lambda x: x[2]*x[3], reverse=True)
  x, y, w, h = Faces[0]

  # Add padding
  padding = 20
  x = max(0, x - padding)
  y = max(0, y - padding)
  w = min(img.shape[1] - x, w + 2*padding)
  h = min(img.shape[0] - y, h + 2*padding)

  # Extract and resize face
  face_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  face = face_rgb[y:y+h, x:x+w]
  face = cv2.resize(face, (160, 160))
  face = face.astype('float32') / 255.0


  return face



In [72]:
def GetData():
    X = []  # lokal definierte Listen
    Y = []

    path_me = "/content/drive/MyDrive/Colab Notebooks/moutia_photos"
    path_others = "/content/drive/MyDrive/Colab Notebooks/others"

    # DEINE Bilder
    for img_file in os.listdir(path_me):
        full_path = os.path.join(path_me, img_file)
        img = cv2.imread(full_path)
        if img is None:
            continue

        face = cropFace(img)  # beachte: crop_face, nicht cropFace
        if face is not None:
            X.append(get_embedding(face))
            Y.append(1)

    # ANDERE Bilder
    for img_file in os.listdir(path_others):
        full_path = os.path.join(path_others, img_file)
        img = cv2.imread(full_path)
        if img is None:
            continue

        face = cropFace(img)
        if face is not None:
            X.append(get_embedding(face))
            Y.append(0)

    # Am Ende in NumPy-Arrays konvertieren
    return np.array(X), np.array(Y)

In [74]:
#Training Classifier to Classify new Images
X, Y = GetData()
X = np.array(X)
Y = np.array(Y)  # zeigt, wie viele pro Klasse
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X, Y)
train_accuracy = clf.score(X, Y)
print(f"✅ Training accuracy: {train_accuracy:.1%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
✅ Training accuracy: 50.0%


In [107]:
img_path = "/content/drive/MyDrive/Colab Notebooks/others/photo-1500648767791-00dcc994a43e.jpg"
face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_default.xml')
img = cv2.imread(img_path)
face = cropFace(img)
if face is not None:
  emb = get_embedding(face)
  pred = clf.predict([emb])
  print(pred[0])
  proba = clf.predict_proba([emb])[0]

  label = "Moutia" if pred == 1 else "SOMEONE ELSE"
  confidence = proba[pred]
  print(f" Result: {label}")
  print(f" Confidence: {confidence:.1%}")

  display_img = img.copy()
  # Find face location again for display
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.1, 5)
  if len(faces) > 0:
    x, y, w, h = faces[0]
    color = (0, 255, 0) if pred == 1 else (0, 0, 255)
    cv2.rectangle(display_img, (x, y), (x+w, y+h), color, 2)
    cv2.putText(display_img, f"{label}: {confidence:.1%}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2_imshow(cv2.resize(display_img, (500, 500)))
else :
  print("Kein Gesicht erkannt!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
0
 Result: SOMEONE ELSE


TypeError: unsupported format string passed to numpy.ndarray.__format__